In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load in mgref file 
# MGREF is the spatial cross-reference files for nonpoint (stationary area)/nonroad and onroad mobile sources.
# https://www.cmascenter.org/smoke/documentation/4.5/manual_smokev45.pdf
file= 'mgref_onroad_MOVES3_04mar2021_nf_v1.75_25.txt'
colheaders= ['FIPS','SCC','Surrogate']
# Including dtype = str here to retain the first columns set of zeros 
df = pd.read_csv(file, skiprows=17, header=None, sep=';', dtype=str)
# Adding column headers to help make a bit more sense. 
df.columns= ['FIPS','SCC','Surrogate']

In [9]:
# SCCs in this file follow the following format:
# First three digits are always 220
# The fourth digit is the Fuel type (01- gas, 02-diesel, 03-CNG, 04- Petroleum gasm 05- Ethonal, 09- EV )
# Fifth and six digits are the vehicle codes. HDVs are 41,42,43,51,52,53,61,62 - Refer to MOVES TSD for specific vehicle types
# 7th and 8th digits are the Road types (01-off-road)
# 9th and 10th are process codes 
# Processes included in file are: 
# 40 - Breakware/tireware, #53- All extended idling exhaust, #62- All Refueling
# #72 - all exhaust and evaporative except refueling and hoteling
# #91 Auxiliary Power Exhaust ,  #92 - Minor Collector urban or rural time #2 - retired 

# Here we want to break the SCC into components to more easily identify the vehicle and road types for HDV ONI.
# First, convert the number to string and index every 2 digits for mobile source, fuel type, etc. 
df['SCC1'] = df['SCC'].astype(str)
df['SCC_mobileSource'] = df['SCC1'].str[0:2]
df['SCC_fuelType'] = df['SCC1'].str[2:4]
df['SCC_VehicleCode'] = df['SCC1'].str[4:6]
df['SCC_RoadType'] = df['SCC1'].str[6:8]
df['SCC_ProcessCode'] = df['SCC1'].str[8:10]

In [10]:
# Look at all process codes for long-haul combination trucks "off road" for idling
# 53: All extended idling exhaust   <- Hotelling 
# 91: Auxilary power exhaust  <- Hotelling
# 62: All refueling
# 72: All exhaust and evaporative except refueling and hotelling  << This is ONI 
# 
df.loc[(df.SCC_VehicleCode.isin(['62'])) & (df['SCC_RoadType'] == '01')]

,FIPS,SCC,Surrogate,SCC1,SCC_mobileSource,SCC_fuelType,SCC_VehicleCode,SCC_RoadType,SCC_ProcessCode
0,000000,2202620153,205,2202620153,22,02,62,01,53
1,000000,2202620191,205,2202620191,22,02,62,01,91
28,000000,2202620162,242,2202620162,22,02,62,01,62
45,000000,2202620172,306,2202620172,22,02,62,01,72
371,008000,2202620153,242,2202620153,22,02,62,01,53
372,008000,2202620191,242,2202620191,22,02,62,01,91
373,023000,2202620153,242,2202620153,22,02,62,01,53
374,023000,2202620191,242,2202620191,22,02,62,01,91
375,034000,2202620153,242,2202620153,22,02,62,01,53
376,034000,2202620191,242,2202620191,22,02,62,01,91


In [11]:
# Next identify which rows contain the values for HDVs AND ONI
# Process 72 being All exhaust and evaporative except refueling and hotelling, or ONI 
df.loc[(df.SCC_VehicleCode.isin(['62'])) & (df['SCC_RoadType'] == '01') & (df['SCC_ProcessCode'] == '72'), 'Surrogate'] = 960

In [12]:
# Next identify which rows contain the values for HDVs AND hotelling
# Select only long haul combiantion trucks, off-road, with all extended idling exhaust and AUX power 
df.loc[(df.SCC_VehicleCode.isin(['62'])) & (df['SCC_RoadType'] == '01') & (df['SCC_ProcessCode'] == '53')|(df['SCC_ProcessCode'] == '91'), 'Surrogate'] = 961

In [13]:
# remove added columns 
df= df[['FIPS','SCC','Surrogate']]

In [14]:
# save as txt file. 
df.to_csv(r'mgref_onroad_MOVES3_04mar2021_nf_cmap_telemetry.txt', index=False,header=None, sep=',', )